# Plotly Interactive Plots

Some interactive plots to visualize the dataset with more details.

In [1]:
# imports

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [2]:
# Test API function before running larger simulations
from examples.sim_trace import generate_trace_api

input_data = {
    "arrival_rate": 0.9, 
    "warm_service_rate": 1/2.016, 
    "cold_service_rate": 1/2.163,
    "expiration_threshold": 600, 
    "max_time": 10000,
}

generate_trace_api(input_data)

{'reqs_cold': 18,
 'reqs_total': 9014,
 'reqs_warm': 8996,
 'prob_cold': 0.0019968937208786333,
 'reqs_reject': 0,
 'prob_reject': 0.0,
 'lifespan_avg': 4350.841761708843,
 'inst_count_avg': 7.490084721949997,
 'inst_running_count_avg': 1.8114269747235925,
 'inst_idle_count_avg': 5.6786577472264055,
 'arrival_rate': 0.9,
 'warm_service_rate': 0.49603174603174605,
 'cold_service_rate': 0.46232085067036527,
 'expiration_threshold': 600,
 'max_time': 10000}

In [12]:
# Plot characteristics for different arrival rates and expiration thresholds
num_arrival_rates = 10
exp_thresholds = [600, 1200, 1800]
exp_threshold_labels = ["10 min", "20 min", "30 min"]
input_data['arrival_rate'] = list(np.repeat(np.logspace(-3,1,num_arrival_rates), len(exp_thresholds)))
input_data['expiration_threshold'] = exp_thresholds * num_arrival_rates
input_data['expiration_threshold_labels'] = exp_threshold_labels * num_arrival_rates

df = pd.DataFrame(data=input_data).reset_index(drop=True)

def generate_trace_apply(x):
    return pd.Series(generate_trace_api(x.to_dict()))

df = df.progress_apply(generate_trace_apply, axis=1)
df.tail()

100%|██████████| 30/30 [00:39<00:00,  1.31s/it]


,reqs_cold,reqs_total,reqs_warm,prob_cold,reqs_reject,prob_reject,lifespan_avg,inst_count_avg,inst_running_count_avg,inst_idle_count_avg,arrival_rate,warm_service_rate,cold_service_rate,expiration_threshold,max_time,expiration_threshold_labels
25,30,35544,35514,0.000844,0,0.0,5903.648933,19.127110,7.230578,11.896532,3.593814,0.496032,0.462321,1200,10000,20 min
26,25,35986,35961,0.000695,0,0.0,4300.195885,18.595133,7.295950,11.299183,3.593814,0.496032,0.462321,1800,10000,30 min
27,60,100308,100248,0.000598,0,0.0,5216.518042,37.313824,20.156791,17.157032,10.000000,0.496032,0.462321,600,10000,10 min
28,59,99644,99585,0.000592,0,0.0,4774.122448,39.226505,20.036503,19.190002,10.000000,0.496032,0.462321,1200,10000,20 min
29,46,100087,100041,0.000460,0,0.0,5738.913739,39.686124,20.180157,19.505967,10.000000,0.496032,0.462321,1800,10000,30 min


In [13]:
# calculate utilizations
df['utilization_percent'] = df['inst_running_count_avg'] / df['inst_count_avg'] * 100

In [22]:
# Plotting imports for a simple interactive plot using plotly express
import plotly.express as px

fig = px.line(df, x='arrival_rate', y='utilization_percent', log_x=True, color='expiration_threshold_labels')
fig.update_layout(xaxis_title="Arrival Rate", yaxis_title="Utilization (%)")
fig.show()